## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [32]:
import glob

import numpy as np
import scipy as sp
import scipy.io
from scipy import signal

# set the sampling rate to 125Hz
fs = 125

# windowing parameters (window_length_s = 8s and window_shift_s = 2s)
window_length_s  = 8  
window_shift_s = 2 

# bpm boundary
lower_bpm, high_bpm = 40, 20

# frequency bins boundary
low, high = lower_bpm/60, high_bpm/60

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estim0tes = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def ground_truth(ref_fl):
    """ loads the ground truth values
    
    args:
        ref_fl: Name of the .mat file that contains reference data
    
    Returns:
        A numpy array of the ground truth values for the input ref_fl
    """
    target = sp.io.loadmat(ref_fl)['BPM0']    
    return target
    

def bandpass_filter(signal, fs, low, high):
    """ 
    filters the input signal between cut-off values.
    
    Args:
        signal: A numpy array representing the signal to filter
        fs: sampling rate
        low = required lower bound of frequency bin
        high = required upper bound of frequency bin
        
    Returns:
        A numpy array of the filtered signal
    """ 
    b, a = sp.signal.butter(3, (low, high), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def feature_generator(ppg, acc, fs):
    """
    uses the filtered signals to generate features within the specified windows

    Args:
        ppg: bandpass filtered ppg signal        
        acc: np.sqrt(accx**2 + accy**2 + accz**2)
            where accx, accy and accz are the bandpass filtered 
            x, y, z, directional components of acc        
        fs: sampling rate
        
    Returns:
        tuple of features derived from the ppg signals
        tuple of features derived from the acc signals
        
        The derived features are 
         1. peaks
         2. frequencies at the peaks
         3. frequency of the signal, and 
         4. the fourier transform of the signal

    """ 

    # fft of ppg signal
    fft_len = 3*len(ppg)
    ppg_freq = np.fft.rfftfreq(fft_len, 1/fs)
    ppg_fft = np.abs(np.fft.rfft(ppg, fft_len))
    
    # Filter fft between low and high frequencies
    ppg_fft[(ppg_freq <= low) & (ppg_freq <= high)]= 0.0
    
    # peaks
    ppg_peaks = sp.signal.find_peaks(ppg_fft, height=2000)[0]
    ppg_peaks_freq = ppg_freq[ppg_peaks]
    
    # fft of acc signal
    acc_freq = ppg_freq
    acc_fft = np.abs(np.fft.rfft(acc, fft_len))
    
    # Filter fft between low and high frequencies
    acc_fft[(acc_freq <= low) & (acc_freq >= high)] = 0.0
    
    # peaks
    acc_peaks = sp.signal.find_peaks(acc_fft, height=None)[0]
    acc_peaks_freq = acc_freq[acc_peaks]
    
    return (ppg_freq, ppg_fft,ppg_peaks, ppg_peaks_freq), (acc_freq, acc_fft, acc_peaks, acc_peaks_freq)


def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Uses the data and reference files, computes pulse rate estimates and returns errors and confidence.
    
    Args:
        data_fl: Data file containing input ppg and accelerometer signals
        ref_fl: refernece file containing ground truth values
    
    Returns:
        Numpy arrays of errors and confidence computed at 8s window
    """
    
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # load the ground truth    
    target = ground_truth(ref_fl)
    
    # filter signals
    ppg = bandpass_filter(ppg, fs, low, high)
    accx = bandpass_filter(accx, fs, low, high)
    accy = bandpass_filter(accy, fs, low, high)
    accz = bandpass_filter(accz, fs, low, high)

    acc = np.sqrt(accx**2 + accy**2 + accz**2)
    
    # compute pulse rates
    window_length = window_length_s * fs
    window_shift = window_shift_s * fs
    
    est, errors, confidence = [],[],[]
   
    for i in range(0, len(ppg) - window_length, window_shift):
        
        features = feature_generator(ppg[i:i+window_length], acc[i:i+window_length], fs)
        ppg_freq, ppg_fft, ppg_peaks, ppg_peaks_freq = features[0]
        acc_freq, acc_fft, acc_peaks, acc_peaks_freq = features[1]
        
        max_ppg_freq = ppg_freq[np.argmax(ppg_fft)]
        max_acc_freq = ppg_freq[np.argmax(acc_fft)]
              
        est_freq = max_ppg_freq

        est.append(est_freq*60)
        
        #compute error and append to the list of errors
        err = np.abs((est-target))
        errors.append(err)
        
        # compute confidence
        window_freq = 30/60
        conf = np.sum(ppg_fft[(ppg_freq > est_freq - window_freq) & (ppg_freq < est_freq + window_freq)])/ np.sum(ppg_fft)                
        confidence.append(conf)    
        
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
 
    return np.array(errors), np.array(confidence)

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

**Code description**
The Evaluate() function will run the code and will load the TROIKA data, estimate the heart rate and then give a confidence interval for each given heart rates. 

**Data description**
The data is made of PPG signal and the accelerometer data in 3 directions.
ECG is the ground truth
We just information about the age of the patients in the dataset (18-25 yo).

**Algorithm description**
The data first go through a bandpass filter to resize the data into the window (40-240 beats/min)
Then, the acelerometer data are aggregated in the 3 directions to get a aggregated magnitude
The signal is divided by 8s windows seprarated by 2 seconds
The heart rate is estimated 
The movement pulse from the heart beat is filtered out using the accelerometer. 

**Algorithm performance**
The Mean Abs Value was used 
The estimated heart rate is compared to the gorund truth (ECG)

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 